In [ ]:
!pip install pyarrow==19.0.0
!pip install -U transformers datasets accelerate bitsandbytes peft gradio

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_ID = "ibm-granite/granite-3.3-2b-instruct"

bnb_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto"
)

In [ ]:
from datasets import Dataset

data = [
    {"instruction": "How to budget?", "response": "Track expenses and save regularly."},
    {"instruction": "Top tax saving tips?", "response": "Invest in ELSS, PPF, etc."},
    {"instruction": "How to track spending?", "response": "Use a daily log or app."}
]

dataset = Dataset.from_list(data)

In [ ]:
def format_chat(example):
    messages = [
        {"role": "system", "content": "You are a friendly finance assistant."},
        {"role": "user", "content": example["instruction"]},
        {"role": "assistant", "content": example["response"]},
    ]
    return {"text": tokenizer.apply_chat_template(messages, tokenize=False)}

dataset = dataset.map(format_chat)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = dataset.map(tokenize_function, batched=True, batch_size=4)
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./personal_finance_chatbot",
    learning_rate=3e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=5,
    save_total_limit=2,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"]
)

In [ ]:
trainer.train()

In [ ]:
import gradio as gr
import time
import random
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import io
import base64

# Set a dark color theme for charts
plt.rcParams['axes.facecolor'] = '#2d2d2d'
plt.rcParams['figure.facecolor'] = '#2d2d2d'
plt.rcParams['grid.color'] = '#444444'
plt.rcParams['axes.grid'] = True
plt.rcParams['font.size'] = 10
plt.rcParams['axes.edgecolor'] = '#666666'
plt.rcParams['text.color'] = '#e0e0e0'
plt.rcParams['axes.labelcolor'] = '#e0e0e0'
plt.rcParams['xtick.color'] = '#e0e0e0'
plt.rcParams['ytick.color'] = '#e0e0e0'

# Generate some sample transaction data for demonstration
def generate_sample_data():
    dates = [datetime.now() - timedelta(days=i) for i in range(30, 0, -1)]
    amounts = [random.randint(5, 100) for _ in range(30)]
    categories = [random.choice(["Food", "Transport", "Entertainment", "Shopping", "Bills"]) for _ in range(30)]
    return pd.DataFrame({"Date": dates, "Amount": amounts, "Category": categories})

# Simulated IBM Watson AI integration functions
def get_ai_response(history, message, user_type, financial_data=None):
    """
    Simulates calling IBM Watson AI services
    In a real implementation, this would connect to IBM Watsonx.ai or Watson Assistant
    """
    time.sleep(1)  # Simulate API call delay

    # Different responses based on user type
    if user_type == "Student":
        responses = {
            "savings": "As a student, consider starting with a high-yield savings account. Many banks offer student accounts with no fees. Aim to save at least 10-15% of any income you receive.",
            "taxes": "Students may be eligible for education tax credits like the American Opportunity Credit. Keep track of your education expenses and speak with a tax professional about Form 1098-T.",
            "investments": "For students, I recommend starting with a Roth IRA if you have earned income. Consider low-cost index funds or ETFs for long-term growth with minimal effort.",
            "budget": "Here's a sample student budget:\n- Housing: 30%\n- Food: 15%\n- Transportation: 10%\n- Education: 25%\n- Personal: 10%\n- Savings: 10%",
            "default": "As a student, it's great that you're thinking about your financial future! Building good habits now will serve you well throughout your life. Consider starting with a simple budget and small, regular savings."
        }
        tone = "encouraging and educational"
    else:
        responses = {
            "savings": "As a professional, you should aim to have 3-6 months of living expenses in an emergency fund. Consider laddering CDs or using high-yield savings accounts for better returns on cash reserves.",
            "taxes": "Professionals should explore tax-advantaged accounts like 401(k)s and HSAs. Consider itemizing deductions if you have significant mortgage interest, state taxes, or charitable contributions.",
            "investments": "For professionals, I recommend a diversified portfolio based on your risk tolerance and time horizon. Consider working with a financial advisor to develop a comprehensive investment strategy.",
            "budget": "Here's a sample professional budget:\n- Housing: 25%\n- Food: 12%\n- Transportation: 10%\n- Debt repayment: 15%\n- Investments: 15%\n- Personal: 13%\n- Savings: 10%",
            "default": "As a professional, comprehensive financial planning is key to achieving your goals. Consider focusing on debt management, retirement planning, and investment diversification."
        }
        tone = "professional and strategic"

    # Check for keywords to provide relevant responses
    message_lower = message.lower()
    if any(word in message_lower for word in ['save', 'savings', 'emergency fund']):
        response = responses['savings']
    elif any(word in message_lower for word in ['tax', 'irs', 'deduct', 'withholding']):
        response = responses['taxes']
    elif any(word in message_lower for word in ['invest', 'stock', 'portfolio', 'return']):
        response = responses['investments']
    elif any(word in message_lower for word in ['budget', 'spending', 'expense']):
        response = responses['budget']
    else:
        response = responses['default']

    # Add a personalized touch
    if financial_data and 'income' in financial_data:
        response += f" Based on your income of ${financial_data['income']:,.2f}, you might consider..."

    return f"[IBM AI Response - {tone}]\n\n{response}"

def generate_budget_summary(income, expenses):
    """Generate an AI-powered budget summary"""
    savings = income - expenses
    savings_rate = (savings / income) * 100 if income > 0 else 0

    if savings_rate > 20:
        assessment = "Excellent savings rate! You're well on your way to financial security."
    elif savings_rate > 10:
        assessment = "Good savings habits. Consider looking for additional opportunities to save."
    else:
        assessment = "Your savings rate could be improved. Let's explore ways to reduce expenses or increase income."

    summary = f"""**AI-Generated Budget Summary**

- Monthly Income: ${income:,.2f}
- Monthly Expenses: ${expenses:,.2f}
- Monthly Savings: ${savings:,.2f}
- Savings Rate: {savings_rate:.1f}%

**Assessment:** {assessment}

**Recommendations:**
1. {"Consider increasing retirement contributions" if savings_rate > 15 else "Build emergency fund first"}
2. {"Review discretionary spending for reduction opportunities" if expenses/income > 0.7 else "Your spending is well-controlled"}
3. {"Explore investment options for your savings" if savings > 500 else "Focus on consistent saving habits"}
"""
    return summary

def analyze_spending(transactions):
    """Analyze spending patterns and provide insights"""
    categories = {}
    for amount, category in transactions:
        categories[category] = categories.get(category, 0) + amount

    total = sum(amount for amount, _ in transactions)
    largest_category = max(categories, key=categories.get)

    insights = f"""**Spending Insights**

- Total Spending: ${total:,.2f}
- Largest Category: {largest_category} (${categories[largest_category]:,.2f})
- Number of Transactions: {len(transactions)}

**Patterns Noticed:**
"""

    if categories.get('Dining', 0) > total * 0.15:
        insights += "- You're spending a significant amount on dining out. Consider meal prepping to save money.\n"
    if categories.get('Entertainment', 0) > total * 0.10:
        insights += "- Entertainment expenses are high. Look for free or low-cost alternatives.\n"
    if categories.get('Utilities', 0) < total * 0.08:
        insights += "- Your utility costs are well managed compared to your overall spending.\n"

    insights += "\n**Suggestions:**\n"
    insights += "1. Set category-specific budgets based on these patterns\n"
    insights += "2. Review subscriptions and recurring payments\n"
    insights += "3. Consider using cash envelopes for discretionary categories"

    return insights

def create_spending_chart(transactions):
    """Create a line chart of spending over time"""
    if not transactions:
        return None

    # Create sample data for demonstration
    df = generate_sample_data()

    # Create the plot
    fig, ax = plt.subplots(figsize=(10, 5))

    # Set background color
    fig.patch.set_facecolor('#2d2d2d')
    ax.set_facecolor('#2d2d2d')

    # Plot spending by category
    colors = ['#7f9aa6', '#9c755f', '#d4a76a', '#b58e70', '#e9c46a']
    for i, category in enumerate(df['Category'].unique()):
        category_data = df[df['Category'] == category]
        ax.plot(category_data['Date'], category_data['Amount'], marker='o', linestyle='-',
                linewidth=2, label=category, color=colors[i % len(colors)])

    # Format the plot
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))
    plt.xticks(rotation=45)
    plt.ylabel('Amount ($)')
    plt.title('Spending Trends Over Time', pad=20)
    plt.legend()
    plt.tight_layout()

    # Convert plot to base64 for embedding in HTML
    buf = io.BytesIO()
    plt.savefig(buf, format='png', facecolor='#2d2d2d')
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    plt.close(fig)

    return f"data:image/png;base64,{img_str}"

def create_category_chart(transactions):
    """Create a pie chart of spending by category"""
    if not transactions:
        return None

    # Create sample data for demonstration
    df = generate_sample_data()

    # Calculate total spending by category
    category_totals = df.groupby('Category')['Amount'].sum()

    # Create the plot
    fig, ax = plt.subplots(figsize=(8, 8))
    fig.patch.set_facecolor('#2d2d2d')

    colors = ['#7f9aa6', '#9c755f', '#d4a76a', '#b58e70', '#e9c46a', '#a87c49']
    wedges, texts, autotexts = ax.pie(category_totals, labels=category_totals.index, autopct='%1.1f%%',
                                      colors=colors, startangle=90)

    # Style the plot
    ax.axis('equal')
    plt.setp(autotexts, size=10, weight="bold", color='white')
    plt.setp(texts, size=10)
    plt.title('Spending by Category', pad=20)

    # Convert plot to base64 for embedding in HTML
    buf = io.BytesIO()
    plt.savefig(buf, format='png', facecolor='#2d2d2d')
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    plt.close(fig)

    return f"data:image/png;base64,{img_str}"

# Main chat function
def respond(message, chat_history, user_type, income, expenses, transactions):
    financial_data = None
    if income > 0:
        financial_data = {'income': income}

    # Get AI response
    response = get_ai_response(chat_history, message, user_type, financial_data)

    # Check if user asked for budget summary
    if any(word in message.lower() for word in ['budget', 'summary', 'overview']):
        if income > 0 and expenses > 0:
            response += "\n\n" + generate_budget_summary(income, expenses)
        else:
            response += "\n\nI'd be happy to generate a budget summary for you! Please provide your monthly income and expenses in the financial information section."

    # Check if user asked for spending analysis
    if any(word in message.lower() for word in ['spending', 'analysis', 'pattern', 'chart', 'graph']):
        if transactions:
            response += "\n\n" + analyze_spending(transactions)
        else:
            response += "\n\nI can analyze your spending patterns! Please add some transactions in the financial information section."

    chat_history.append((message, response))
    return "", chat_history

def clear_chat():
    return []

def update_transaction_display(transactions):
    if not transactions:
        return "No transactions added yet"

    display_text = "**Your Transactions:**\n\n"
    for i, (amount, category) in enumerate(transactions, 1):
        display_text += f"{i}. ${amount:,.2f} - {category}\n"

    return display_text

def update_analytics(transactions):
    spending_chart = create_spending_chart(transactions)
    category_chart = create_category_chart(transactions)

    if spending_chart and category_chart:
        analytics_html = f"""
        <div style="display: flex; flex-direction: column; gap: 20px;">
            <div style="background: #3d3d3d; padding: 15px; border-radius: 10px; box-shadow: 0 4px 6px rgba(0,0,0,0.1); border: 1px solid #555555;">
                <h3 style="margin-top: 0; color: #7f9aa6;">Spending Trends</h3>
                <img src="{spending_chart}" style="width: 100%;">
            </div>
            <div style="background: #3d3d3d; padding: 15px; border-radius: 10px; box-shadow: 0 4px 6px rgba(0,0,0,0.1); border: 1px solid #555555;">
                <h3 style="margin-top: 0; color: #7f9aa6;">Spending by Category</h3>
                <img src="{category_chart}" style="width: 100%;">
            </div>
        </div>
        """
    else:
        analytics_html = """
        <div style="text-align: center; padding: 40px; background: #3d3d3d; border-radius: 10px; border: 1px solid #555555;">
            <h3 style="color: #7f9aa6;">No data available yet</h3>
            <p>Add transactions to see spending analytics</p>
        </div>
        """

    return analytics_html

# Demo data for initial transaction list
demo_transactions = [
    (45.67, "Dining"),
    (120.50, "Groceries"),
    (89.99, "Entertainment"),
    (65.00, "Transportation"),
    (150.00, "Utilities")
]

# Custom CSS for a dark theme
custom_css = """
:root {
    --primary: #7f9aa6;
    --secondary: #9c755f;
    --accent: #d4a76a;
    --background: #1e1e1e;
    --text: #e0e0e0;
    --card: #2d2d2d;
    --border: #444444;
}

body {
    background: var(--background) !important;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    color: var(--text) !important;
}

.gr-button {
    background: linear-gradient(to bottom, #7f极速aa6, #5a7986) !important;
    color: white !important;
    border: none !important;
    border-radius: 8px !important;
    padding: 10px 20px !important;
    font-weight: 600 !important;
    box-shadow: 0 2极速4px rgba(0,0,0,0.2) !important;
}

.gr-button:hover {
    background: linear-gradient(to bottom, #6a8996, #4a6774) !important;
    box-shadow: 0 4px 8px rgba(0,0,0,0.3) !important;
}

.gr-chatbot {
    background: var(--card) !important;
    border-radius: 12px !important;
    box-shadow: 0 4px 12px rgba(0,0,0,0.1) !important;
    border: 1px solid var(--border) !important;
}

.gr-textbox {
    border-radius: 8px !important;
    border: 1px solid var(--border) !important;
    background: #3d3d3d !important;
    color: var(--text) !important;
}

.gr-radio {
    padding: 10px !important;
    background: var(--card) !important;
    border-radius: 8px !important;
    border: 极速1px solid var(--border) !important;
}

.gr-number {
    border-radius: 8px !important;
    border: 1px solid var(--border) !important;
    background: #3d3d3d !important;
    color: var(--text) !important;
}

.gr-dropdown {
    border-radius: 8px !important;
    border: 1px solid var(--border) !important;
    background: #3d3d3d !important;
    color: var(--text) !important;
}

.dashboard-card {
    background: var(--card);
    border-radius: 12px;
    padding: 20px;
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);
    border: 1px solid var(--border);
    margin-bottom: 20px;
}

.dashboard-card h3 {
    margin-top: 0;
    color: var(--primary);
    border-bottom: 1px solid var(--border);
    padding-bottom: 10px;
}

.analytics-container {
    background: var(--card);
    border-radius: 12px;
    padding: 20px;
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);
    border: 1px极速 solid var(--border);
}

.welcome-container {
    text-align: center;
    padding: 40px 20px;
    background: linear-gradient(135deg, #2d2d2d 0%, #252525 100%);
    border-radius: 16px;
    border: 1px solid var(--border);
    margin: 20px 0;
}

.welcome-title {
    color: var(--primary);
    font-size: 2.5rem;
    margin-bottom: 10px;
}

.welcome-subtitle {
    color: var(--accent);
    font-size: 1.2rem;
    margin-bottom: 30px;
}

.feature-grid {
    display: grid;
    grid-template-columns: repeat(2, 1fr);
    gap: 20px;
    margin: 30px 0;
}

.feature-item {
    background: #3d3极速d3d;
    padding: 20px;
    border-radius: 12px;
    text-align: center;
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);
    border: 1px solid var(--border);
}

.feature-icon {
    font-size: 2rem;
    margin-bottom: 15px;
    color: var(--accent);
}

.get-started-btn {
    background: linear-gradient(to bottom, #7f9aa6, #5a7986) !important;
    color: white !important;
    border: none !important;
    border-radius: 8px !important;
    padding: 15px 30px !important;
    font-size: 1.1rem !important;
    font-weight: 600 !important;
    margin-top: 20px;
    box-shadow: 0 4px 8px rgba(0,0,0,0.2) !important;
}

.get-started-btn:hover {
    background: linear-gradient(to bottom, #6a8996, #4a6774) !important;
    box-shadow: 0 6px 12px rgba(0,0,0,0.3) !important;
}

.coin-icon {
    font-size: 3rem;
    margin-bottom: 20px;
    color: var(--accent);
}

.gr-markdown {
    color: var(--text) !important;
}

.gr-label {
    color: var(--text) !important;
}
"""

# Create welcome page
def create_welcome():
    return """
    <div class="welcome-container">
        <div class="coin-icon">🪙</div>
        <h1 class="welcome-title">Welcome to Coins</h1>
        <p class="welcome-subtitle">Your personal finance assistant powered by IBM Watson AI</p>

        <p>Coins helps you manage your finances with AI-powered insights, budget tracking, and personalized advice tailored to your financial situation.</p>

        <div class="feature-grid">
            <div class="feature-item">
                <div class="feature-icon">💬</div>
                <h3>AI Chat Assistant</h3>
                <p>Get personalized financial advice using IBM Watson AI</p>
            </div>
            <div class="feature-item">
                <div class="feature-icon">📊</div>
                <h3>Budget Analytics</h3>
                <p>Visualize your spending patterns with interactive charts</p>
            </div>
            <div class="feature-item">
                <div class="feature-icon">💰</div>
                <h3>Savings Goals</h3>
                <p>Set and track progress toward your financial objectives</p>
            </div>
            <div class="feature-item">
                <div class="feature-icon">📈</div>
                <h3>Investment Advice</h3>
                <p>Get recommendations tailored to your risk profile</p>
            </div>
        </div>

        <p>Ready to take control of your finances? Click the button below to get started!</p>
    </div>
    """

# Create the main interface
with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue"), css=custom_css) as demo:
    gr.Markdown("# 🪙 Coins - Personal Finance Assistant")

    # Create a state to track if we're on the welcome page or main interface
    show_welcome = gr.State(value=True)

    # Welcome page
    with gr.Column(visible=True) as welcome_col:
        gr.HTML(create_welcome())
        get_started_btn = gr.Button("Get Started", elem_classes="get-started-btn")

    # Main interface (initially hidden)
    with gr.Column(visible=False) as main_col:
        with gr.Row():
            with gr.Column(scale=1):
                with gr.Column(elem_classes="dashboard-card"):
                    gr.Markdown("### 👤 User Profile")
                    user_type = gr.Radio(
                        ["Student", "Professional"],
                        value="Professional",
                        label="Select your profile"
                    )

                with gr.Column(elem_classes="dashboard-card"):
                    gr.Markdown("### 💰 Financial Information")
                    income = gr.Number(
                        value=4500,
                        label="Monthly Income ($)",
                        info="Enter your approximate monthly income"
                    )
                    expenses = gr.Number(
                        value=3200,
                        label="Monthly Expenses ($)",
                        info="Enter your approximate monthly expenses"
                    )

                with gr.Column(elem_classes="dashboard-card"):
                    gr.Markdown("### ➕ Add Transaction")
                    with gr.Row():
                        trans_amount = gr.Number(label="Amount ($)", value=35)
                        trans_category = gr.Dropdown(
                            ["Housing", "Utilities", "Groceries", "Dining", "Transportation",
                             "Entertainment", "Healthcare", "Education", "Other"],
                            label="Category",
                            value="Dining"
                        )
                    add_trans_btn = gr.Button("Add Transaction", elem_classes="gr-button")
                    transactions = gr.State(value=demo_transactions)
                    trans_display = gr.Markdown()

                    # Update transaction display initially
                    demo_trans_display = update_transaction_display(demo_transactions)
                    trans_display.value = demo_trans_display

                    clear_btn = gr.Button("Clear All Data", variant="secondary")

            with gr.Column(scale=2):
                chatbot = gr.Chatbot(
                    value=[(None, "Hello! I'm Coins, your Personal Finance Assistant powered by IBM Watson AI. I can help you with savings, taxes, investments, and more. How can I assist you today?")],
                    height=400,
                    elem_classes="gr-chatbot"
                )
                msg = gr.Textbox(
                    placeholder="Ask me about savings, taxes, investments, or type 'budget summary'...",
                    label="Your Message"
                )
                with gr.Row():
                    submit_btn = gr.Button("Send", elem_classes="gr-button")
                    clear_chat_btn = gr.Button("Clear Conversation", variant="secondary")

                # Analytics section
                gr.Markdown("### 📊 Spending Analytics")
                analytics = gr.HTML()
                # Initialize with empty analytics
                analytics.value = update_analytics([])

    # Function to toggle between welcome and main interface
    def toggle_interface(show_welcome):
        show_welcome = not show_welcome
        return [
            gr.update(visible=show_welcome),
            gr.update(visible=not show_welcome),
            show_welcome
        ]

    # Connect the get started button
    get_started_btn.click(
        fn=toggle_interface,
        inputs=show_welcome,
        outputs=[welcome_col, main_col, show_welcome]
    )

    # Transaction handling
    def add_transaction(amount, category, current_transactions):
        if amount is None or amount <= 0 or not category:
            return current_transactions, trans_display.value, analytics.value
        new_transactions = current_transactions + [(amount, category)]
        display_text = update_transaction_display(new_transactions)
        analytics_html = update_analytics(new_transactions)
        return new_transactions, display_text, analytics_html

    add_trans_btn.click(
        add_transaction,
        inputs=[trans_amount, trans_category, transactions],
        outputs=[transactions, trans_display, analytics]
    )

    # Chat functionality
    def chat_response(message, chat_history, user_type, income, expenses, transactions):
        # This wrapper ensures we always return the updated chat history
        _, updated_history = respond(message, chat_history, user_type, income, expenses, transactions)
        return "", updated_history

    msg.submit(
        chat_response,
        inputs=[msg, chatbot, user_type, income, expenses, transactions],
        outputs=[msg, chatbot]
    )

    submit_btn.click(
        chat_response,
        inputs=[msg, chatbot, user_type, income, expenses, transactions],
        outputs=[msg, chatbot]
    )

    # Clear functions
    clear_chat_btn.click(
        lambda: [],
        inputs=None,
        outputs=chatbot
    )

    def clear_financial_data():
        empty_analytics = """
        <div style="text-align: center; padding: 40px; background: #3d3d3d; border-radius: 10px; border: 1px solid #555555;">
            <h3 style="color: #7f9aa6;">No data available yet</h3>
            <p>Add transactions to see spending analytics</p>
        </div>
        """
        return 0, 0, [], "No transactions added yet", empty_analytics

    clear_btn.click(
        clear_financial_data,
        inputs=None,
        outputs=[income, expenses, transactions, trans_display, analytics]
    )

if __name__ == "__main__":
    demo.launch()